In [1]:
!pip install catboost
!pip install scikit-learn --upgrade

     |████████████████████████████████| 67.4 MB 49 kB/s 
     |████████████████████████████████| 22.3 MB 4.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


# Dataframes

In [2]:
# If you have installation questions, please reach out
import seaborn as sns
import pandas as pd # data storage
import catboost as cats # graident boosting 

from catboost import CatBoostRegressor, Pool

import numpy as np  # math and stuff
import matplotlib.pyplot as plt # plotting utility
import sklearn # ML and stats

print('catboost ver:', cats.__version__)
print('scikit ver:', sklearn.__version__)

import datetime

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import max_error, mean_squared_error, median_absolute_error

catboost ver: 0.26.1
scikit ver: 0.24.2


In [3]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS0_Merged_dataset_imputed_08_23_2021.csv')

df2 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS1_Merged_dataset_imputed_08_23_2021.csv')

df3 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS2_Merged_dataset_imputed_08_23_2021.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

print(df.columns.values) # printing all column names

df.describe()

['depth_ft' 'USGS_ID' 'CAL' 'GR' 'DT' 'SP' 'DENS' 'PE' 'RESD' 'PHIN'
 'PHID' 'PE_smooth' 'GR_smooth' 'gz_linear_interp' 'gz_pchip_interp'
 'perm_air_md' 'perm_klink_md' 'He_por' 'horz_perm_md' 'effective_por'
 'por_percent' 'stress_swanson_perm_md' 'sample' 'Na' 'Na Err' 'Mg'
 'Mg Err' 'Al' 'Al Err' 'Si' 'Si Err' 'P' 'P Err' 'S' 'S Err' 'K' 'K Err'
 'Ca' 'Ca Err' 'Ti' 'Ti Err' 'Cr' 'Cr Err' 'Mn' 'Mn Err' 'Fe' 'Fe Err'
 'Co' 'Co Err' 'Ni' 'Ni Err' 'Cu' 'Cu Err' 'Zn' 'Zn Err' 'Ba' 'Ba Err']


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.468406,0.325999,0.430844,0.811259,0.613371,0.351642,1.011018,0.172737,0.172737,0.351702,0.326029,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.191316,0.237404,0.263514,0.162843,0.214604,0.186912,1.503608,0.226664,0.226664,0.178735,0.229901,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.000000,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.360072,0.122318,0.174146,0.743878,0.424460,0.216157,-0.037336,0.004595,0.004595,0.220800,0.131058,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.275825,0.471657,0.867010,0.605516,0.329327,0.795008,0.005532,0.005532,0.337049,0.283192,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530980,0.662807,0.925012,0.830179,0.470706,1.378387,0.332503,0.332503,0.463317,0.525448,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

In [5]:
df = df[df.He_por >= 0]

df = df[df.USGS_ID != 'E997']

In [6]:
dataset = df[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth','He_por'
]]

In [7]:
dataset.replace('NaN',np.nan, regex=True, inplace=True)# 
#dataset = dataset.dropna()
np.shape(dataset)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


(357, 12)

In [8]:
X = dataset[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

Y = dataset[['He_por']]

Y_array = np.array(Y.values)

In [9]:
seed = 42 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

In [10]:
preds = []
preds2 = []

def catboost_he_por(X_train, X_test, y_train, y_test, export_name='TEST', max_iter = 200):
  model = CatBoostRegressor(objective='RMSE', iterations=max_iter)

  model.fit(X_train, y_train, verbose=max_iter )

  preds = model.predict(X_test)

  rmse = mean_squared_error(y_test, preds, squared=False)
  print("Root Mean Squared Error: %f" % (rmse))
  max = max_error(y_test, preds)
  print("Max Error: %f" % (max))
  MAE = median_absolute_error(y_test, preds)
  print("Median Abs Error: %f" % (MAE))

  grid = {'learning_rate': [ 0.05, 0.1, 0.2, 0.3],
        'depth': [ 2, 4, 6, 8, 10],
        'l2_leaf_reg': [1, 2, 3, 4, 5, 7]}

  model_grid = CatBoostRegressor(objective='RMSE', 
                                 iterations=max_iter, 
                                 verbose=False)

  # Grid Search
  grid_search_result = model_grid.grid_search(grid, 
                                            X=X_train, 
                                            y=y_train, 
                                            cv=5,
                                            verbose=False)
  
  model2 = CatBoostRegressor(objective='RMSE',
                           depth=grid_search_result['params']['depth'],
                           l2_leaf_reg=grid_search_result['params']['l2_leaf_reg'],
                           learning_rate=grid_search_result['params']['learning_rate'],
                           iterations=max_iter)

  model2.fit(X_train, y_train, verbose=500 )

  preds2 = model2.predict(X_test)

  rmse2 = mean_squared_error(y_test, preds2, squared=False)
  print("Root Mean Squared Error: %f" % (rmse2))
  max2 = max_error(y_test, preds2)
  print("Max Error: %f" % (max2))
  MAE2= median_absolute_error(y_test, preds2)
  print("Median Abs Error: %f" % (MAE2))

  x = datetime.datetime.now()

  d = {'target': [Y.columns.values, Y.columns.values],
     'MSE': [rmse, rmse2],
     'MAE': [MAE, MAE2],
     'MaxError': [max, max2], 
     'iter':[max_iter, max_iter],
     'day': [x.day, x.day], 
     'month':[x.month, x.month], 
     'year':[x.year, x.year],
     'model':['catboost', 'catboost'],
     'version':[cats.__version__, cats.__version__ ]}
 
  filepath = 'drive/My Drive/1_lewis_research/analysis/experiments/catboost/catboost_results/'

  results = pd.DataFrame(data=d)

  results.to_csv(filepath+export_name)

  return results

# Iterations

In [11]:
iter = 250

#Offset 0

In [12]:
catboost_he_por(X_train, X_test, y_train, y_test,'OS0_por_cat.csv', iter)

Learning rate set to 0.088488
0:	learn: 1.8305738	total: 54.1ms	remaining: 13.5s
249:	learn: 0.4084589	total: 933ms	remaining: 0us
Root Mean Squared Error: 1.977440
Max Error: 6.111462
Median Abs Error: 0.735670

bestTest = 1.565874484
bestIteration = 235


bestTest = 1.537453096
bestIteration = 174


bestTest = 1.450125548
bestIteration = 87


bestTest = 1.621591809
bestIteration = 139


bestTest = 1.586922401
bestIteration = 242


bestTest = 1.504840932
bestIteration = 162


bestTest = 1.522671841
bestIteration = 106


bestTest = 1.540544781
bestIteration = 59


bestTest = 1.573682193
bestIteration = 249


bestTest = 1.540012964
bestIteration = 156


bestTest = 1.444953007
bestIteration = 127


bestTest = 1.517337576
bestIteration = 108


bestTest = 1.573236715
bestIteration = 238


bestTest = 1.526911959
bestIteration = 241


bestTest = 1.45610791
bestIteration = 142


bestTest = 1.514175891
bestIteration = 58


bestTest = 1.600336711
bestIteration = 238


bestTest = 1.553106732
bes

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[He_por],1.977440,0.735670,6.111462,250,23,8,2021,catboost,0.26.1
1,[He_por],1.920989,0.697805,5.958920,250,23,8,2021,catboost,0.26.1


# Offset 1

In [13]:
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df2 = df2[df2.He_por >= 0]
df2 = df2[df2.USGS_ID != 'E997'] # removing E997

dataset2 = df2[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth','He_por']]

# Features we will use for prediction
X2 = dataset2[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y = dataset2[['He_por']]

Y_array2 = np.array(Y.values)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2.values, Y_array2, test_size=test_size)

In [14]:
catboost_he_por(X_train2, X_test2, y_train2, y_test2,'OS1_por_cat2.csv', iter)

Learning rate set to 0.088488
0:	learn: 1.8107742	total: 4ms	remaining: 996ms
249:	learn: 0.3487992	total: 811ms	remaining: 0us
Root Mean Squared Error: 1.753633
Max Error: 5.642539
Median Abs Error: 0.595588

bestTest = 1.365663226
bestIteration = 249


bestTest = 1.351988521
bestIteration = 158


bestTest = 1.308408043
bestIteration = 85


bestTest = 1.288816689
bestIteration = 66


bestTest = 1.31785958
bestIteration = 249


bestTest = 1.291730184
bestIteration = 200


bestTest = 1.348195193
bestIteration = 94


bestTest = 1.405906665
bestIteration = 51


bestTest = 1.410938079
bestIteration = 249


bestTest = 1.296229625
bestIteration = 210


bestTest = 1.30185777
bestIteration = 101


bestTest = 1.226083874
bestIteration = 65


bestTest = 1.431585845
bestIteration = 249


bestTest = 1.23690651
bestIteration = 230


bestTest = 1.350011927
bestIteration = 98


bestTest = 1.219725107
bestIteration = 95


bestTest = 1.427227825
bestIteration = 248


bestTest = 1.261447453
bestIteratio

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[He_por],1.753633,0.595588,5.642539,250,23,8,2021,catboost,0.26.1
1,[He_por],1.864200,0.816273,5.688028,250,23,8,2021,catboost,0.26.1


# Offset 2

In [15]:
df3 = df3.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df3 = df3[df3.He_por >= 0]
df3 = df3[df3.USGS_ID != 'E997'] # removing E997

dataset3 = df3[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth','He_por']]

# Features we will use for prediction
X3 = dataset3[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y3 = dataset3[['He_por']]

Y_array3 = np.array(Y3.values)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3.values, Y_array3, test_size=test_size)

In [16]:
catboost_he_por(X_train3, X_test3, y_train3, y_test3,'OS2_por_cat2.csv', iter)

Learning rate set to 0.088488
0:	learn: 1.8714361	total: 3.13ms	remaining: 778ms
249:	learn: 0.3593508	total: 914ms	remaining: 0us
Root Mean Squared Error: 1.490842
Max Error: 5.296719
Median Abs Error: 0.736927

bestTest = 1.373358257
bestIteration = 232


bestTest = 1.250193082
bestIteration = 242


bestTest = 1.315228685
bestIteration = 162


bestTest = 1.282903537
bestIteration = 125


bestTest = 1.360849376
bestIteration = 248


bestTest = 1.223036118
bestIteration = 242


bestTest = 1.251215647
bestIteration = 247


bestTest = 1.31271285
bestIteration = 99


bestTest = 1.413093245
bestIteration = 248


bestTest = 1.229949326
bestIteration = 249


bestTest = 1.353474607
bestIteration = 143


bestTest = 1.246314248
bestIteration = 75


bestTest = 1.38039621
bestIteration = 244


bestTest = 1.241719384
bestIteration = 238


bestTest = 1.34799675
bestIteration = 195


bestTest = 1.255615065
bestIteration = 73


bestTest = 1.440808115
bestIteration = 248


bestTest = 1.281205269
bestI

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[He_por],1.490842,0.736927,5.296719,250,23,8,2021,catboost,0.26.1
1,[He_por],1.437098,0.631690,4.612079,250,23,8,2021,catboost,0.26.1
